## Imports and Load + Split Data

In [8]:
from __future__ import print_function
import argparse
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data.sampler import SubsetRandomSampler
import os

train_dataset = datasets.MNIST('../data', train=True, download=True,
            transform=transforms.Compose([      
                transforms.ToTensor(),           
                transforms.Normalize((0.1307,), (0.3081,))
            ]))

test_dataset = datasets.MNIST('../data', train=False,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ]))

subset_indices_train = np.load('indices_train.npy')
subset_indices_valid = np.load('indices_valid.npy')

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=64,
    sampler=SubsetRandomSampler(subset_indices_train)
)
val_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=64,
    sampler=SubsetRandomSampler(subset_indices_valid)
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64, shuffle=True)


## Default ConvNet with No Data Augmentation

In [24]:
class ConvNet(nn.Module):
    '''
    Design your model with convolutional layers.
    '''
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=1)
        self.conv2 = nn.Conv2d(8, 8, 3, 1)
        self.dropout1 = nn.Dropout2d(0.5)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(200, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout2(x)

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        output = F.log_softmax(x, dim=1)
        return output

def test(model, test_loader):
    model.eval()    
    test_loss = 0
    correct = 0
    test_num = 0
    with torch.no_grad(): 
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.view_as(pred)).sum().item()
            test_num += len(data)
    test_loss /= test_num
    print('\nAccuracy: {}/{} ({:.0f}%)\n'.format( correct, test_num, 100. * correct / test_num))
    
model = ConvNet()
model.load_state_dict(torch.load('default_ConvNet.pt'))

print("Training set with no data augmentation")
test(model, train_loader)
print("Validation set with no data augmentation")
test(model, val_loader)

Training set with no data augmentation

Accuracy: 49807/51005 (98%)

Validation set with no data augmentation

Accuracy: 8797/8995 (98%)



## Default ConvNet with Data Augmentation (Random Perspective)

In [25]:
train_dataset_aug = datasets.MNIST('../data', train=True, download=True,
            transform=transforms.Compose([    
                transforms.RandomPerspective(distortion_scale=0.5, p=0.5),
                transforms.ToTensor(),           
                transforms.Normalize((0.1307,), (0.3081,))
            ]))

subset_indices_train = np.load('indices_train.npy')
subset_indices_valid = np.load('indices_valid.npy')

train_loader_aug = torch.utils.data.DataLoader(
    train_dataset_aug, batch_size=64,
    sampler=SubsetRandomSampler(subset_indices_train)
)
val_loader_aug = torch.utils.data.DataLoader(
    train_dataset_aug, batch_size=64,
    sampler=SubsetRandomSampler(subset_indices_valid)
)

model = ConvNet()
model.load_state_dict(torch.load('default_ConvNet.pt'))

print("Training set with data augmentation")
test(model, train_loader_aug)
print("Validation set with data augmentation")
test(model, val_loader_aug)

Training set with data augmentation

Accuracy: 38807/51005 (76%)

Validation set with data augmentation

Accuracy: 6850/8995 (76%)

